# 케라스(keras)로 나만의 데이터셋을 이용하여 얼굴 인식 학습하기

## 모델 설계

In [17]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), strides=1, input_shape=(96, 96, 3)))
model.add(layers.Conv2D(64, (5, 5), activation='relu', strides=2, ))
model.add(layers.Conv2D(128, (5, 5), strides=1, ))
model.add(layers.Conv2D(128, (5, 5), activation='relu', strides=2, ))
model.add(layers.Conv2D(256, (5, 5), strides=1, ))
model.add(layers.Conv2D(256, (5, 5), activation='relu', strides=2, ))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 92, 92, 64)        4864      
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 44, 44, 64)        102464    
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 40, 40, 128)       204928    
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 18, 18, 128)       409728    
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 14, 14, 256)       819456    
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 5, 5, 256)         1638656   
_________________________________________________________________
flatten_4 (Flatten)          (None, 6400)              0         
__________

- layer (Conv2D와 MaxPooling2D)
    - 입력
        - input_shape(image_height, image_width, image_channels)
    - 출력
        - (height, width, channels) 크기의 3D 텐서를 return
        - height와 width는 컨볼루션, stride의 영향으로 작아짐-
        - channels는 입력에 의해
- Dense
    - 입력
        - 1D의 형태이기 때문에, 위에서 만든 3D 텐서를 1D로 평평하게 해주어야 합니다.(Flatten)
    - 출력
        - 마지막 Dense Layer의 출력이 분류(Classification)하고자 하는 크기!

## 데이터셋 만들기 (이미지 불러오기, 레이블 정보 만들어서 가져오기, 가공하기)

In [23]:
batch_size = 256
epochs = 40

def get_input_queue(csv_file_name, num_epochs = epochs):
    train_images = []
    train_labels = []
    for line in open(csv_file_name,'r'):
        cols = re.split(',|\n',line)
        train_images.append(cols[0])
        train_labels.append(int(cols[1]))
    input_queue = tf.train.slice_input_producer([train_images,train_labels],
                                               num_epochs = num_epochs,shuffle = True)
    return input_queue

def read_data(input_queue):
    image_file = input_queue[0]
    label = input_queue[1]
    image =  tf.image.decode_jpeg(tf.read_file(image_file),channels=image_color)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    label = tf.image.convert_image_dtype(label, dtype=tf.int64)
    image.set_shape((96, 96, 3))
    return image, label #,image_file

def read_data_batch(csv_file_name, batch_size=batch_size):
    input_queue = get_input_queue(csv_file_name)
    image, label = read_data(input_queue)
    #image,label,file_name= read_data(input_queue)

    # random image
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image,max_delta=0.5)
    image = tf.image.random_contrast(image,lower=0.2,upper=2.0)
    image = tf.image.random_hue(image,max_delta=0.08)
    image = tf.image.random_saturation(image,lower=0.2,upper=2.0)
    # batch_image,batch_label,batch_file = tf.train.batch([image,label,file_name],batch_size=batch_size)
        
    batch_image,batch_label = tf.train.batch([image,label],batch_size=batch_size)
    #,enqueue_many=True)
    #batch_file = tf.reshape(batch_file,[batch_size,1])
    #batch_label_on_hot=tf.one_hot(tf.to_int64(batch_label), num_classes, on_value=1.0, off_value=0.0)
    #return batch_image, batch_label_on_hot#, batch_file
    
    return batch_image, batch_label

batch_images, batch_labels = read_data_batch('training_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'training_data.csv'